In [325]:
def parse_line(l):
    name, job = l.split(": ")
    
    if job.isnumeric():
        out = int(job)
        return {name: {"out":out}}
    else:
        job = job.replace("/", "//").split(" ")
        prev = (job[0], job[2])
        job = f"a {job[1]} b"
        return {name:{"job":job, "prev":prev}}


input = [parse_line(l) for l in open("inputs/21-input.txt").read().splitlines()]
monkeys = {k: v for l in input.copy() for k, v in l.items()}

done = {}

while "root" not in done:
    done.update({m:int(d["out"]) for m,d in monkeys.items() if "out" in d.keys()})

    for m,d in monkeys.items():
        if m not in done:
            a,b = d["prev"]
            if a in done and b in done:
                #d["prev"] = None
                a = monkeys[a]["out"]
                b = monkeys[b]["out"]
                d["out"] = eval(d["job"])

done["root"]

63119856257960

In [335]:
import re

monkeys["root"]["job"] = "a == b"

#
def update_job(m):
    if "job" in monkeys[m].keys():
        job = monkeys[m]["job"]
        prev = monkeys[m]["prev"]
        new_job = job.replace("a", update_job(prev[0])).replace("b", update_job(prev[1]))
        return str(int(eval(new_job))) if "humn" not in new_job else f"({new_job})"
            
    elif m == "humn":
        return m
    else:
        return str(monkeys[m]["out"])

# find x where a and b are known
def inverse(equation, a, b):
    m = {
        "a = x + b":  f"{a} -  {b}",
        "a = x - b":  f"{a} +  {b}",
        "a = x * b":  f"{a} // {b}",
        "a = x // b": f"{a} *  {b}",
        "a = b + x":  f"{a} -  {b}",
        "a = b - x":  f"{b} -  {a}",
        "a = b * x":  f"{a} // {b}",
        "a = b // x": f"{b} // {a}",
    }
    return eval(m[equation])

# find inverse of each bracketed expression in turn '(...) + a == b'
def solve(calc):
    l, r = calc.split(" == ")

    a, expr = (l,r) if l.isnumeric() else (r,l)
    
    # start or end with number 
    start = re.search("^(\d+) ([\/|\*|\+|-]+) (.+)", expr[1:-1])
    end = re.search("(.+) ([\/|\*|\+|-]+) (\d+)$",expr[1:-1])

    if start:
        b, op, x = start.group(1,2,3)
        eq = f"b {op} x"
    elif end:
        b, op, x = end.group(3,2,1)
        eq = f"x {op} b"
    else:
        return int(a)
    
    eq = "a = " + eq

    return solve(f"{x} == {inverse(eq, a, b)}")
    
# Produce final calculation in terms of unknown value `humn` 
calc = update_job("root")[1:-1]
# Solve for `humn`
solve(calc)

3006709232464